<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-1.csv'
url2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-2.csv'
url3 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-3.csv'
url4 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-4.csv'
url5 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/14-5.csv'

data_1 = pd.read_csv(url1)
data_2 = pd.read_csv(url2)
data_3 = pd.read_csv(url3)
data_4 = pd.read_csv(url4)
data_5 = pd.read_csv(url5)

data = data_1
data = data.append(data_2)
data = data.append(data_3)
data = data.append(data_4)
data = data.append(data_5)

data = data.reset_index()
data = data.drop(['index'], axis=1)

In [0]:
# [1]

from sklearn.neighbors import KNeighborsClassifier
import math

data1 = data[(data['Class'].isna()) & (data['Amount'] > 200)]
nonnull = data[data['Class'].isna() == False]

X_cols = list(data1.columns)[1:29]
train_X = nonnull[X_cols]
train_y = nonnull['Class']
test_X = data1[X_cols]

# preparing to give closest 5 points in eucledian
knc = KNeighborsClassifier(n_neighbors=5, p=2)
model = knc.fit(train_X, train_y)

# finding 5 points per test sample (28,704 X 5)
neig_dist, neig_index = model.kneighbors(test_X[X_cols])

# creating test_y (28,704 X 1)
test_y = pd.DataFrame(neig_index, columns=range(1, 6))

def getClass(x):
    # if at least 1 point is 1, make the class as 1
    for i in range(1, 6):
        if train_y[x[i]] == 1:
            return 1
    return 0

test_y = test_y.assign(esti=test_y.apply(lambda x: getClass(x), axis=1))

# filling the calculated class in the original data
test_X.reset_index(inplace=True)
test = test_X.assign(eClass=test_y['esti'])
test.set_index('index', inplace=True)

# applying estimation in the original null records
data1 = data1.assign(Class=test['eClass'])

# merging Null records into NonNull records
merged_df = pd.concat([data1, nonnull], axis=0, ignore_index=False)

max_val = merged_df[merged_df['Class'] == 1]['Amount'].max()
min_val = merged_df[merged_df['Class'] == 1]['Amount'].min()

print(f'Answer [1] : Range = {math.floor(max_val - min_val)}')

Answer [1] : Range = 25691


In [0]:
# [2]

from scipy import stats
import math

group0 = merged_df[merged_df['Class'] == 0]['V2']
group1 = merged_df[merged_df['Class'] == 1]['V2']

t_val, p_val = stats.ttest_ind(group0, group1, equal_var=False)

print(f'Answer [2] : t-value = {math.floor(t_val)}')

Answer [2] : t-value = 22


In [0]:
# [3]

from sklearn.linear_model import LinearRegression
import math

data3 = merged_df[merged_df['Class'] == 1]
train_X = data3[X_cols]
train_y = data3['Amount']

lr = LinearRegression()
model = lr.fit(train_X, train_y)
score = model.score(train_X, train_y)

print(f'Answer [3] : R-squared = {math.floor(score * 100) / 100}')

Answer [3] : R-squared = 0.93


In [0]:
# [4]

from sklearn.linear_model import LogisticRegression

train = merged_df[merged_df['Time'] <= 100000]
test = merged_df[merged_df['Time'] > 100000]

train_X = train[X_cols + ['Amount']]
train_y = train['Class']
test_X = test[X_cols + ['Amount']]
test_y = test['Class']

lr = LogisticRegression(C=100000, random_state=1234, solver='newton-cg')
model = lr.fit(train_X, train_y)
pred = model.predict_proba(test_X)

pred_df = pd.DataFrame(pred, columns=['0', '1'])
pred_df = pred_df.assign(esti=pred_df.apply(lambda x: 1 if x['1'] >= 0.4 else 0,
                                            axis=1))
pred_df = pred_df.assign(fact=test_y.reset_index()['Class'])
pred_df = pred_df.assign(accurate=pred_df.apply(lambda x:
                                                1 if x['esti'] == x['fact'] else 0,
                                                axis=1))
accuracy = pred_df['accurate'].sum() / pred_df['accurate'].count()

print(f'Answer [4] : Accuracy = {math.floor(accuracy * 1000) / 1000}')

Answer [4] : Accuracy = 0.815
